<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [23]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-e3c7c5953a85dc99'
PROJECT = 'qwiklabs-gcp-e3c7c5953a85dc99'
REGION = 'us-central1'

In [24]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [25]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [47]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [48]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bq.Query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").execute().result().to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,5896567601480310696,342825
1,9068386407968572094,330863
2,260598435387740869,320332
3,2700219941840496344,307395
4,454960867574323744,346515


In [74]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.000452"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.000452"
traindf = bq.Query(trainQuery).execute().result().to_dataframe()
evaldf = bq.Query(evalQuery).execute().result().to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 12052 examples in the train dataset and 3035 in the eval dataset


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [75]:
import pandas as pd
def preprocess(df):
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]

  nous = df.copy(deep=True)
  #nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'

  twins_etc = dict(zip([1,2,3,4,5],
                     ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  return pd.concat([df, nous])

In [76]:
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.856376,False,34,Single(1),36.0,5742197815970064689
1,8.187968,True,31,Single(1),40.0,3408502330831153141
2,9.812775,False,37,Single(1),41.0,5742197815970064689
3,8.313632,True,36,Single(1),38.0,7604198770453299557
4,8.437091,False,34,Single(1),40.0,328012383083104805


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [77]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [78]:
%bash
wc -l *.csv
head *.csv
tail *.csv

   6022 eval.csv
  23910 train.csv
  29932 total
==> eval.csv <==
6.1729433359999994,True,24,Single(1),36.0,1088037545023002395
7.1870697412,False,27,Single(1),39.0,7146494315947640619
6.4992274837599995,True,36,Single(1),31.0,6244544205302024223
7.24879917456,True,24,Single(1),41.0,6782146986770280327
7.826410301,True,35,Single(1),38.0,7146494315947640619
6.93794738514,False,26,Single(1),39.0,6910174677251748583
8.50102482272,True,33,Single(1),40.0,8904940584331855459
8.24969784404,True,28,Single(1),39.0,270792696282171059
10.06189763768,False,35,Single(1),40.0,7517141034410775575
5.732018812,False,34,Single(1),34.0,1639186255933990135

==> train.csv <==
6.8563763481999995,False,34,Single(1),36.0,5742197815970064689
8.18796841068,True,31,Single(1),40.0,3408502330831153141
9.81277528162,False,37,Single(1),41.0,5742197815970064689
8.313631900019999,True,36,Single(1),38.0,7604198770453299557
8.437090766739999,False,34,Single(1),40.0,328012383083104805
8.2452885988,True,31,Single(1),39.0,

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License